In [ ]:
#코엔엘파이(KoNLPy)는 한국어 자연어 처리를 위한 형태소 분석기 패키지입니다.
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 6.6 MB/s 
     |████████████████████████████████| 453 kB 41.8 MB/s 


In [ ]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 22.05 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-08-11 03:51:07--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 18.205.93.2, 18.205.93.1, 18.205.93.0, ...
Connecting to bitbucket.org (bitbucket.org)|18.205.93.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eu

In [ ]:
#시각화툴
!pip install pyldavis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=48572bc6fb627f629ac55030338d14713548974f052f312823ed9c56cc838773
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyldavis


In [ ]:
import numpy as np
import pandas as pd
import warnings # 경고 메시지 무시
warnings.filterwarnings(action='ignore')
# 한국어 형태소 분석기 중 성능이 가장 우수한 Mecab 사용
from konlpy.tag import Mecab
mecab = Mecab()
from tqdm import tqdm # 작업 프로세스 시각화
import re # 문자열 처리를 위한 정규표현식 패키지
from gensim import corpora # 단어 빈도수 계산 패키지
import gensim # LDA 모델 활용 목적
import pyLDAvis.gensim_models # LDA 시각화용 패키지
from collections import Counter # 단어 등장 횟수 카운트

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import os
path = '/content/gdrive/MyDrive/subjects_after/'
file_list_hippo_py = []
for (root, dirs, files) in os.walk(path):
  if len(files) > 0:
    for file_name in files:
      if "prpr" in file_name:    
        file_list_hippo_py.append(root+'/'+file_name)
documents = []
for i in file_list_hippo_py:
  a=''
  for line in open((i),"r").readlines():
      if line and not line.replace(':','').replace(' ', '').strip().isdecimal():
        a=a+line.strip()
      else:
        if a!='':
          documents.append(a)
          a=''
  documents.append(a)
        
documents

['타조가 알을 품고 있었습니다. 그중 한 개가 대굴대굴 댁대구르르……굴러갔습니다.낮잠을 자던 하마, 푸루푸루는 떠들썩한 말소리에 눈을 떴습니다.“아유, 하마씨. 축하해요. 큰 알이네요. 곧 엄마가 되겠군요.”“틀림없이 당신을 닮은 예쁜 아기가 태어날 거예요.”“몸조심하세요.”푸루푸루는 동물 친구들이 무슨 말을 하는 것인지 전혀 알 수 없었습니다.',
 '낮잠을 자던 하마, 푸루푸루는 떠들썩한 말소리에 눈을 떴습니다.“아유, 하마씨. 축하해요. 큰 알이네요. 곧 엄마가 되겠군요.”“틀림없이 당신을 닮은 예쁜 아기가 태어날 거예요.”“몸조심하세요.”푸루푸루는 동물 친구들이 무슨 말을 하는 것인지 전혀 알 수 없었습니다.푸루푸루는 어여차 하고 일어섰습니다.“아니, 이게 뭐지……? 아까 그 아주머니들, 이 알을 보고 한 말이었구나.”푸루푸루는 그 알이, 자기가 낳은 것으로 완전히 믿게 되었습니다.',
 '“푸루푸루, 어디 가 있었어? 산책 나가서 돌아오지 않길래 얼마나 걱정했다고.”우두머리 하마가 말했습니다. 푸루푸루는 누가 말을 붙여도 고개만 내저을 뿐 대답하지 못했습니다. 모두 화를 내고 말았습니다.자, 이제부터가 큰일입니다. 먹을 수도 없고, 말을 할 수도 없습니다. 혼자 있는 날이 계속되었습니다.',
 '자, 이제부터가 큰일입니다. 먹을 수도 없고, 말을 할 수도 없습니다. 혼자 있는 날이 계속되었습니다.어느 날, 삑-삑 울음소리가 나더니, 눈이 큰 아기가 톡, 알을 꺠고 나왔습니다.“성공이다!”푸루푸루는 한껏 큰 소리로 외쳤습니다.',
 '어느 날, 삑-삑 울음소리가 나더니, 눈이 큰 아기가 톡, 알을 꺠고 나왔습니다.“성공이다!”푸루푸루는 한껏 큰 소리로 외쳤습니다.하마는 헤엄을 칠 줄 알아야 합니다. 푸루푸루는 아기에게 헤엄을 가르치기로 했습니다. 그런데 이 아기는 물을 싫어하는 것 같았어요. 이런 아기 하마는 처음 봅니다. 푸루푸루는 걱정이 되었어요. 큰 눈, 기다란 목, 가는 다리.“날 전혀 닮지 않았어. 새들한테서 깃털을 받았기 때문인가……?”',

In [ ]:
#데이터 전처리
hippo_df = pd.DataFrame({'document':documents})
hippo_df

,document
0,타조가 알을 품고 있었습니다. 그중 한 개가 대굴대굴 댁대구르르……굴러갔습니다.낮잠...
1,"낮잠을 자던 하마, 푸루푸루는 떠들썩한 말소리에 눈을 떴습니다.“아유, 하마씨. 축..."
2,"“푸루푸루, 어디 가 있었어? 산책 나가서 돌아오지 않길래 얼마나 걱정했다고.”우두..."
3,"자, 이제부터가 큰일입니다. 먹을 수도 없고, 말을 할 수도 없습니다. 혼자 있는 ..."
4,"어느 날, 삑-삑 울음소리가 나더니, 눈이 큰 아기가 톡, 알을 꺠고 나왔습니다.“..."
5,하마는 헤엄을 칠 줄 알아야 합니다. 푸루푸루는 아기에게 헤엄을 가르치기로 했습니다...
6,"“하마씨, 뭘 하고 있는 거예요?”펠리컨 아주머니였어요.“알은 배의 보드라운 털로 ..."
7,"“여기요, 여기가 가장 좋겠어요.”입을 아-앙 벌리게 하고, 둥지를 만들었습니다.“..."
8,"“그러고 보니까 하마씨를 닮지는 않았지만, 걱정할 것 없어요.”엄마 개구리가 말했습..."
9,"”맞아요, 역시 내 아기예요. 이제 틀림없이 헤엄도 치게 될 거예요.”푸루푸루는, ..."


In [ ]:
#데이터 탐색 
#데이터셋에서 전반적으로 결측치 존재 여부, 데이터 타입, 데이터 개수를 확인합니
hippo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   document  19 non-null     object
dtypes: object(1)
memory usage: 280.0+ bytes


In [ ]:
# 한글 외 텍스트 제거
#숫자, 특수문자, 영문 등  관련된 의미를 추출해 내기 어려운 모든 문자열을 제거합니다.
_removed = list(map(lambda text: re.sub('[^가-힣 ]', '', text), hippo_df['document']))
hippo_df['document'] = _removed
hippo_df

,document
0,타조가 알을 품고 있었습니다 그중 한 개가 대굴대굴 댁대구르르굴러갔습니다낮잠을 자던...
1,낮잠을 자던 하마 푸루푸루는 떠들썩한 말소리에 눈을 떴습니다아유 하마씨 축하해요 큰...
2,푸루푸루 어디 가 있었어 산책 나가서 돌아오지 않길래 얼마나 걱정했다고우두머리 하마...
3,자 이제부터가 큰일입니다 먹을 수도 없고 말을 할 수도 없습니다 혼자 있는 날이 계...
4,어느 날 삑삑 울음소리가 나더니 눈이 큰 아기가 톡 알을 꺠고 나왔습니다성공이다푸루...
5,하마는 헤엄을 칠 줄 알아야 합니다 푸루푸루는 아기에게 헤엄을 가르치기로 했습니다 ...
6,하마씨 뭘 하고 있는 거예요펠리컨 아주머니였어요알은 배의 보드라운 털로 따뜻하게 품...
7,여기요 여기가 가장 좋겠어요입을 아앙 벌리게 하고 둥지를 만들었습니다깃털이 따뜻하니...
8,그러고 보니까 하마씨를 닮지는 않았지만 걱정할 것 없어요엄마 개구리가 말했습니다내 ...
9,맞아요 역시 내 아기예요 이제 틀림없이 헤엄도 치게 될 거예요푸루푸루는 눈이 왕방울...


In [ ]:
hippo_tokenized = list(map(lambda text: mecab.nouns(text), hippo_df['document']))
hippo_tokenized

[['타조',
  '알',
  '중',
  '개',
  '댁',
  '구르',
  '르',
  '낮잠',
  '하마',
  '푸루푸루',
  '말',
  '소리',
  '눈',
  '축하',
  '알',
  '엄마',
  '당신',
  '아기',
  '거',
  '몸',
  '조심',
  '푸루푸루',
  '동물',
  '친구',
  '말',
  '것',
  '수'],
 ['낮잠',
  '하마',
  '푸루푸루',
  '말',
  '소리',
  '눈',
  '축하',
  '알',
  '엄마',
  '당신',
  '아기',
  '거',
  '몸',
  '조심',
  '푸루푸루',
  '동물',
  '친구',
  '말',
  '것',
  '수',
  '푸루푸루',
  '이게',
  '뭐',
  '아주머니',
  '알',
  '말',
  '푸루푸루',
  '알',
  '자기',
  '것'],
 ['푸루푸루',
  '어디',
  '산책',
  '걱정',
  '우두머리',
  '하마',
  '말',
  '푸루푸루',
  '누가',
  '말',
  '고개',
  '뿐',
  '대답',
  '화',
  '이제',
  '큰일',
  '수',
  '말',
  '수',
  '날',
  '계속'],
 ['이제',
  '큰일',
  '수',
  '말',
  '수',
  '날',
  '계속',
  '날',
  '울음',
  '소리',
  '눈',
  '아기',
  '성공',
  '푸루푸루',
  '소리'],
 ['날',
  '울음',
  '소리',
  '눈',
  '아기',
  '성공',
  '푸루푸루',
  '소리',
  '하마',
  '헤엄',
  '줄',
  '푸루푸루',
  '아기',
  '헤엄',
  '아기',
  '물',
  '것',
  '아기',
  '하마',
  '푸루푸루',
  '걱정',
  '눈',
  '목',
  '다리',
  '날',
  '새',
  '깃털',
  '때문'],
 ['하마',
  '헤엄',
  '줄',
  '푸루푸루',
  '아기',
  '헤엄',

꾸루꾸루와 푸루푸루가 고유명사로 처리되지 않는 문제점 발생 
-> mecab에 고유명사로 추가 

In [ ]:
%cd /content/

/content


In [ ]:
!ls

gdrive			     mecab-ko-dic-2.1.1-20180720	 sample_data
mecab-0.996-ko-0.9.2	     mecab-ko-dic-2.1.1-20180720.tar.gz
mecab-0.996-ko-0.9.2.tar.gz  Mecab-ko-for-Google-Colab


In [ ]:
%cd mecab-ko-dic-2.1.1-20180720/

/content/mecab-ko-dic-2.1.1-20180720


In [ ]:
!ls

aclocal.m4	EP.csv	     Makefile.am     Person-actor.csv	user-nnp.csv
AUTHORS		ETM.csv      Makefile.in     Person.csv		user-person.csv
autogen.sh	ETN.csv      matrix.bin      Place-address.csv	user-place.csv
ChangeLog	feature.def  matrix.def      Place.csv		VA.csv
char.bin	Foreign.csv  missing	     Place-station.csv	VCN.csv
char.def	Group.csv    MM.csv	     pos-id.def		VCP.csv
clean		Hanja.csv    model.bin	     Preanalysis.csv	VV.csv
CoinedWord.csv	IC.csv	     model.def	     README		VX.csv
config.log	Inflect.csv  NEWS	     rewrite.def	Wikipedia.csv
config.status	INSTALL      NNBC.csv	     right-id.def	XPN.csv
configure	install-sh   NNB.csv	     Symbol.csv		XR.csv
configure.ac	J.csv	     NNG.csv	     sys.dic		XSA.csv
COPYING		left-id.def  NNP.csv	     tools		XSN.csv
dicrc		MAG.csv      NorthKorea.csv  unk.def		XSV.csv
EC.csv		MAJ.csv      NP.csv	     unk.dic
EF.csv		Makefile     NR.csv	     user-dic


In [ ]:
!ls user-dic

nnp.csv  person.csv  place.csv	README.md


In [ ]:
import pandas as pd

nnp_df = pd.read_csv("./user-dic/nnp.csv")
nnp_df

,대우,Unnamed: 1,Unnamed: 2,Unnamed: 3,NNP,*,F,대우.1,*.1,*.2,*.3,*.4,*.5
0,구글,NaN,NaN,NaN,NNP,*,T,구글,*,*,*,*,*
1,푸루푸루,NaN,NaN,NaN,NNP,*,F,푸루푸루,*,*,*,*,*
2,꾸루꾸루,NaN,NaN,NaN,NNP,*,F,꾸루꾸루,*,*,*,*,*


In [ ]:
word_list = ["푸루푸루", "꾸루꾸루"]


In [ ]:
#종성여부를 판단하는 라이브러리
!pip install jamo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from jamo import h2j, j2hcj

#종성여부를 판단하는 함수

def get_jongsung_TF(sample_text):
    sample_text_list = list(sample_text)
    last_word = sample_text_list[-1]
    last_word_jamo_list = list(j2hcj(h2j(last_word)))
    last_jamo = last_word_jamo_list[-1]

    jongsung_TF = "T"

    if last_jamo in ['ㅏ', 'ㅑ', 'ㅓ', 'ㅕ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ', 'ㅘ', 'ㅚ', 'ㅙ', 'ㅝ', 'ㅞ', 'ㅢ', 'ㅐ,ㅔ', 'ㅟ', 'ㅖ', 'ㅒ']:
        jongsung_TF = "F"

    return jongsung_TF

In [ ]:
with open("./user-dic/nnp.csv", 'r', encoding='utf-8') as f:
  file_data = f.readlines()

word_list = ["푸루푸루", "꾸루꾸루"]

for word in word_list:
  jongsung_TF = get_jongsung_TF(word)

  line = '{},,,,NNP,*,{},{},*,*,*,*,*\n'.format(word, jongsung_TF, word)

  file_data.append(line)

In [ ]:
file_data

['대우,,,,NNP,*,F,대우,*,*,*,*,*\n',
 '구글,,,,NNP,*,T,구글,*,*,*,*,*\n',
 '푸루푸루,,,,NNP,*,F,푸루푸루,*,*,*,*,*\n',
 '꾸루꾸루,,,,NNP,*,F,꾸루꾸루,*,*,*,*,*\n',
 '푸루푸루,,,,NNP,*,F,푸루푸루,*,*,*,*,*\n',
 '꾸루꾸루,,,,NNP,*,F,꾸루꾸루,*,*,*,*,*\n']

In [ ]:
with open("./user-dic/nnp.csv", 'w', encoding='utf-8') as f:
  for line in file_data:
    f.write(line)

In [ ]:
with open("./user-dic/nnp.csv", 'r', encoding='utf-8') as f:
  file_new = f.readlines()
file_new

['대우,,,,NNP,*,F,대우,*,*,*,*,*\n',
 '구글,,,,NNP,*,T,구글,*,*,*,*,*\n',
 '푸루푸루,,,,NNP,*,F,푸루푸루,*,*,*,*,*\n',
 '꾸루꾸루,,,,NNP,*,F,꾸루꾸루,*,*,*,*,*\n',
 '푸루푸루,,,,NNP,*,F,푸루푸루,*,*,*,*,*\n',
 '꾸루꾸루,,,,NNP,*,F,꾸루꾸루,*,*,*,*,*\n']

In [ ]:
#tools 디렉토리 확인 
!ls 

aclocal.m4	EP.csv	     Makefile.am     Person-actor.csv	user-nnp.csv
AUTHORS		ETM.csv      Makefile.in     Person.csv		user-person.csv
autogen.sh	ETN.csv      matrix.bin      Place-address.csv	user-place.csv
ChangeLog	feature.def  matrix.def      Place.csv		VA.csv
char.bin	Foreign.csv  missing	     Place-station.csv	VCN.csv
char.def	Group.csv    MM.csv	     pos-id.def		VCP.csv
clean		Hanja.csv    model.bin	     Preanalysis.csv	VV.csv
CoinedWord.csv	IC.csv	     model.def	     README		VX.csv
config.log	Inflect.csv  NEWS	     rewrite.def	Wikipedia.csv
config.status	INSTALL      NNBC.csv	     right-id.def	XPN.csv
configure	install-sh   NNB.csv	     Symbol.csv		XR.csv
configure.ac	J.csv	     NNG.csv	     sys.dic		XSA.csv
COPYING		left-id.def  NNP.csv	     tools		XSN.csv
dicrc		MAG.csv      NorthKorea.csv  unk.def		XSV.csv
EC.csv		MAJ.csv      NP.csv	     unk.dic
EF.csv		Makefile     NR.csv	     user-dic


In [ ]:
#add-userdic.sh* 쉘 스크립트 확인 
!ls tools


add-userdic.sh	convert_for_using_store.sh  mecab-bestn.sh


In [ ]:
#해당 스크립트 실행
!bash ./tools/add-userdic.sh

generating userdic...
nnp.csv
/content/mecab-ko-dic-2.1.1-20180720/tools/../model.def is not a binary model. reopen it as text mode...
reading /content/mecab-ko-dic-2.1.1-20180720/tools/../user-dic/nnp.csv ... 
done!
person.csv
/content/mecab-ko-dic-2.1.1-20180720/tools/../model.def is not a binary model. reopen it as text mode...
reading /content/mecab-ko-dic-2.1.1-20180720/tools/../user-dic/person.csv ... 
done!
place.csv
/content/mecab-ko-dic-2.1.1-20180720/tools/../model.def is not a binary model. reopen it as text mode...
reading /content/mecab-ko-dic-2.1.1-20180720/tools/../user-dic/place.csv ... 
done!
test -z "model.bin matrix.bin char.bin sys.dic unk.dic" || rm -f model.bin matrix.bin char.bin sys.dic unk.dic
/usr/local/libexec/mecab/mecab-dict-index -d . -o . -f UTF-8 -t UTF-8
reading ./unk.def ... 13
emitting double-array: 100% |###########################################| 
reading ./XSN.csv ... 124
reading ./Group.csv ... 3176
reading ./XPN.csv ... 83
reading ./Person-actor

In [ ]:
#make install 을 통해서 추가 완료 
!make install

make[1]: Entering directory '/content/mecab-ko-dic-2.1.1-20180720'
make[1]: Nothing to be done for 'install-exec-am'.
 /bin/mkdir -p '/usr/local/lib/mecab/dic/mecab-ko-dic'
 /usr/bin/install -c -m 644 model.bin matrix.bin char.bin sys.dic unk.dic left-id.def right-id.def rewrite.def pos-id.def dicrc '/usr/local/lib/mecab/dic/mecab-ko-dic'
make[1]: Leaving directory '/content/mecab-ko-dic-2.1.1-20180720'


제대로 분석이 되는 것을 알 수 있다. 

In [ ]:
from konlpy.tag import Mecab

mecab = Mecab()

for word in word_list:
  print(mecab.pos(word))
  

[('푸루푸루', 'NNP')]
[('꾸루꾸루', 'NNP')]


In [ ]:
mecab.pos("안녕 포티투마루 푸루푸루 꾸루꾸루")

[('안녕', 'IC'),
 ('포', 'MAG'),
 ('티투', 'NNP'),
 ('마루', 'NNP'),
 ('푸루푸루', 'NNP'),
 ('꾸루꾸루', 'NNP')]

문제점  
1. 음성을 번역할 때 푸르푸르를 푸르푸르, 푸르포로, 포로포로 등 다양한 어휘로 번역함 -> 통일 필요

2. 한글자인 토큰은 의미 없는 경우가 많음  

3. 불용어를 넣어줄 필요가 있음 

### 전처리용 딕셔너리 load

### 불용어 사전 리스트

주제와는 무관하지만 빈출되는 단어인 불용어는 미리 제거하는 작업이 필요합니다.

In [ ]:
#불용어 사전 리스트
stopword_list = pd.read_excel('/content/gdrive/MyDrive/Colab Notebooks/natural_language/stopword_list.xlsx')
stopword_list.head()

,stopword
0,가까스로
1,가량
2,가령
3,가민
4,가민커넥트


### 데이터 치환 리스트


*  같은 의미로 사용된 여러 데이터를 특정 단어로 통일화하기 위한 데이터셋입니다.
* 특정 단어의 빈출여부가 핵심이기 때문에 같은 의미의 단어를 통일화하는 게 중요합니다. 

In [ ]:
replace_list = pd.read_excel('/content/gdrive/MyDrive/Colab Notebooks/natural_language/replace_list.xlsx')
replace_list.head()

,before_replacement,after_replacement
0,프로푸르,푸루푸루
1,푸르푸르,푸루푸루
2,푸르푸루,푸루푸루
3,프로포로,푸루푸루
4,프로포르,푸루푸루


### 한 글자 키워드 리스트


*  일반적으로 토큰이 1글자인 경우는 대부분 불용어에 해당하기 때문에 제거합니다.
* 단, 프로젝트 성격에 따라 1글자의 단어가 핵심 키워드가 될 수 있습니다.
*  아래의 리스트는 이처럼 키워드로 생각되는 1글자 단어를 리스트업 하였습니다.





In [ ]:
one_char_keyword = pd.read_excel('/content/gdrive/MyDrive/Colab Notebooks/natural_language/one_char_list.xlsx')
one_char_keyword.head()

,one_char_keyword
0,알
1,몸
2,물
3,입
4,말


# 데이터 치환
같은 의미의 단어를 하나의 단어로 통일하는 작업입니다.
LDA 토픽 모델링은 빈출 어휘를 중심으로 결과를 제공하기 때문에 단어를 통일할 필요가 있습니다.

In [ ]:
def replace_word(text):
    for i in range(len(replace_list['before_replacement'])):
        try:
            # 치환할 단어가 있는 경우에만 데이터 치환 수행
            if replace_list['before_replacement'][i] in text:
                text = text.replace(replace_list['before_replacement'][i], replace_list['after_replacement'][i])
        except Exception as e:
            print(f"Error 발생 / 에러명: {e}")
    return text

In [ ]:
hippo_df['document_after'] = ''
text_replaced_list = []
for text in tqdm(hippo_df['document']):
    text_replaced = replace_word(str(text)) # 문자열 데이터 변환
    text_replaced_list.append(text_replaced)
hippo_df['document_after'] = text_replaced_list
hippo_df

100%|██████████| 19/19 [00:00<00:00, 1038.42it/s]


,document,document_after
0,타조가 알을 품고 있었습니다 그중 한 개가 대굴대굴 댁대구르르굴러갔습니다낮잠을 자던...,타조가 알을 품고 있었습니다 그중 한 개가 대굴대굴 댁대구르르굴러갔습니다낮잠을 자던...
1,낮잠을 자던 하마 푸루푸루는 떠들썩한 말소리에 눈을 떴습니다아유 하마씨 축하해요 큰...,낮잠을 자던 하마 푸루푸루는 떠들썩한 말소리에 눈을 떴습니다아유 하마씨 축하해요 큰...
2,푸루푸루 어디 가 있었어 산책 나가서 돌아오지 않길래 얼마나 걱정했다고우두머리 하마...,푸루푸루 어디 가 있었어 산책 나가서 돌아오지 않길래 얼마나 걱정했다고우두머리 하마...
3,자 이제부터가 큰일입니다 먹을 수도 없고 말을 할 수도 없습니다 혼자 있는 날이 계...,자 이제부터가 큰일입니다 먹을 수도 없고 말을 할 수도 없습니다 혼자 있는 날이 계...
4,어느 날 삑삑 울음소리가 나더니 눈이 큰 아기가 톡 알을 꺠고 나왔습니다성공이다푸루...,어느 날 삑삑 울음소리가 나더니 눈이 큰 아기가 톡 알을 꺠고 나왔습니다성공이다푸루...
5,하마는 헤엄을 칠 줄 알아야 합니다 푸루푸루는 아기에게 헤엄을 가르치기로 했습니다 ...,하마는 헤엄을 칠 줄 알아야 합니다 푸루푸루는 아기에게 헤엄을 가르치기로 했습니다 ...
6,하마씨 뭘 하고 있는 거예요펠리컨 아주머니였어요알은 배의 보드라운 털로 따뜻하게 품...,하마씨 뭘 하고 있는 거예요펠리컨 아주머니였어요알은 배의 보드라운 털로 따뜻하게 품...
7,여기요 여기가 가장 좋겠어요입을 아앙 벌리게 하고 둥지를 만들었습니다깃털이 따뜻하니...,여기요 여기가 가장 좋겠습니다입을 아앙 벌리게 하고 둥지를 만들었습니다깃털이 따뜻하...
8,그러고 보니까 하마씨를 닮지는 않았지만 걱정할 것 없어요엄마 개구리가 말했습니다내 ...,그러고 보니까 하마씨를 닮지는 않았지만 걱정할 것 안됩니다엄마 개구리가 말했습니다내...
9,맞아요 역시 내 아기예요 이제 틀림없이 헤엄도 치게 될 거예요푸루푸루는 눈이 왕방울...,맞아요 역시 내 아기예요 이제 틀림없이 헤엄도 치게 될 거예요푸루푸루는 눈이 왕방울...


# 토큰화

1. KoNLPy에서 속도 및 토큰화 측면에서 성능이 가장 우수한 Mecab 형태소 분석기 활용
2. 명사가 문장 내 맥락을 파악하는 데 핵심 형태소이며 빈출 어휘를 쉽게 파악하기 위해 명사만 추출

In [ ]:
hippo_tokenized = list(map(lambda text: mecab.morphs(text), hippo_df['document_after']))
hippo_tokenized

[['타조',
  '가',
  '알',
  '을',
  '품',
  '고',
  '있',
  '었',
  '습니다',
  '그',
  '중',
  '한',
  '개',
  '가',
  '대굴대굴',
  '댁',
  '대',
  '구르',
  '르',
  '굴러갔',
  '습니다',
  '낮잠',
  '을',
  '자',
  '던',
  '하마',
  '푸루푸루',
  '는',
  '떠들썩',
  '한',
  '말',
  '소리',
  '에',
  '눈',
  '을',
  '떴',
  '습니다',
  '아유',
  '하마',
  '씨',
  '축하',
  '해요',
  '큰',
  '알',
  '이',
  '네요',
  '곧',
  '엄마',
  '가',
  '되',
  '겠',
  '군요',
  '틀림없이',
  '당신',
  '을',
  '닮',
  '은',
  '예쁜',
  '아기',
  '가',
  '태어날',
  '거',
  '예요',
  '몸',
  '조심',
  '하',
  '세요',
  '푸루푸루',
  '는',
  '동물',
  '친구',
  '들',
  '이',
  '무슨',
  '말',
  '을',
  '하',
  '는',
  '것',
  '인지',
  '전혀',
  '알',
  '수',
  '없',
  '었',
  '습니다'],
 ['낮잠',
  '을',
  '자',
  '던',
  '하마',
  '푸루푸루',
  '는',
  '떠들썩',
  '한',
  '말',
  '소리',
  '에',
  '눈',
  '을',
  '떴',
  '습니다',
  '아유',
  '하마',
  '씨',
  '축하',
  '해요',
  '큰',
  '알',
  '이',
  '네요',
  '곧',
  '엄마',
  '가',
  '되',
  '겠',
  '군요',
  '틀림없이',
  '당신',
  '을',
  '닮',
  '은',
  '예쁜',
  '아기',
  '가',
  '태어날',
  '거',
  '예요',
  '몸',
  '조심',
  '하',
  '세요'

# 불용어 제거
토큰화 후 토큰이 2글자인 경우 불용어에 해당되지 않는 토큰만 분석에 활용합니다. 토큰이 1글자인 경우에는 1글자 키워드 리스트에 포함되는 토큰만 분석에 활용합니다.

In [ ]:
def remove_stopword(tokens):
    text_removed_stopword = []
    for token in tokens:
        # 토큰의 글자 수가 2글자 이상인 경우
        if 1 < len(token):
            # 토큰이 불용어가 아닌 경우만 분석용 리뷰 데이터로 포함
            if token not in list(stopword_list['stopword']):
                text_removed_stopword.append(token)
        # 토큰의 글자 수가 1글자인 경우
        else:
            # 1글자 키워드에 포함되는 경우만 분석용 리뷰 데이터로 포함
            if token in list(one_char_keyword['one_char_keyword']):
                text_removed_stopword.append(token)
    return text_removed_stopword

In [ ]:
text_removed_stopword = list(map(lambda tokens : remove_stopword(tokens),hippo_tokenized))
text_removed_stopword

[['타조',
  '알',
  '품',
  '대굴대굴',
  '구르',
  '굴러갔',
  '낮잠',
  '하마',
  '푸루푸루',
  '떠들썩',
  '말',
  '소리',
  '아유',
  '하마',
  '축하',
  '알',
  '엄마',
  '군요',
  '틀림없이',
  '예쁜',
  '아기',
  '태어날',
  '몸',
  '조심',
  '세요',
  '푸루푸루',
  '동물',
  '말',
  '인지',
  '전혀',
  '알'],
 ['낮잠',
  '하마',
  '푸루푸루',
  '떠들썩',
  '말',
  '소리',
  '아유',
  '하마',
  '축하',
  '알',
  '엄마',
  '군요',
  '틀림없이',
  '예쁜',
  '아기',
  '태어날',
  '몸',
  '조심',
  '세요',
  '푸루푸루',
  '동물',
  '말',
  '인지',
  '전혀',
  '알',
  '푸루푸루',
  '어여차',
  '일어섰',
  '아까',
  '아주머니',
  '알',
  '말',
  '푸루푸루',
  '알',
  '낳',
  '완전히'],
 ['푸루푸루',
  '산책',
  '나가',
  '돌아오',
  '않',
  '길래',
  '걱정',
  '우두머리',
  '하마',
  '말',
  '푸루푸루',
  '말',
  '붙여',
  '고개',
  '내저',
  '대답',
  '못했',
  '화',
  '말',
  '큰일',
  '먹',
  '말'],
 ['큰일',
  '먹',
  '말',
  '삑삑',
  '울음',
  '소리',
  '더니',
  '아기',
  '알',
  '꺠고',
  '나왔',
  '성공',
  '푸루푸루',
  '한껏',
  '소리',
  '외쳤'],
 ['삑삑',
  '울음',
  '소리',
  '더니',
  '아기',
  '알',
  '꺠고',
  '나왔',
  '성공',
  '푸루푸루',
  '한껏',
  '소리',
  '외쳤',
  '하마',
  '헤엄',
  '알',
  '푸루푸루',
  '아기',

# LDA 토픽 모델링

## 하이퍼파라미터 튜닝

In [ ]:
NUM_TOPICS = 10 # 토픽 개수는 하이퍼파라미터
# passes: 딥러닝에서 Epoch와 같은 개념으로, 전체 corpus로 모델 학습 횟수 결정
PASSES = 100

## 2) 모델 학습 함수

In [ ]:
def lda_modeling(text_prep):
    # 단어 인코딩 및 빈도수 계산
    dictionary = corpora.Dictionary(text_prep)
    corpus = [dictionary.doc2bow(text) for text in text_prep]
    # LDA 모델 학습
    model = gensim.models.ldamodel.LdaModel(corpus, 
                                            num_topics = NUM_TOPICS, 
                                            id2word = dictionary, 
                                            passes = PASSES)
    return model, corpus, dictionary

## 3) 토픽별 단어 구성 출력 함수

토픽별로 구성된 단어의 비율을 출력하는 함수입니다.

In [ ]:
def print_topic_prop(topics, RATING):
    topic_values = []
    for topic in topics:
        topic_value = topic[1]
        topic_values.append(topic_value)
    topic_prop = pd.DataFrame({"topic_num" : list(range(1, NUM_TOPICS + 1)), "word_prop": topic_values})
    topic_prop.to_excel('/content/gdrive/MyDrive/result_nlp/' + RATING +  '.xlsx')
    display(topic_prop)

## 4) 시각화 함수

시각화 함수는 result 폴더 내에 저장합니다

In [ ]:
def lda_visualize(model, corpus, dictionary, RATING):
    pyLDAvis.enable_notebook()
    result_visualized = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
    pyLDAvis.display(result_visualized)
    # 시각화 결과 저장
    RESULT_FILE = '/content/gdrive/MyDrive/result_nlp/' + RATING + '.html'
    pyLDAvis.save_html(result_visualized, RESULT_FILE)

In [ ]:
model, corpus, dictionary = lda_modeling(text_removed_stopword)
NUM_WORDS = 10 # 토픽별 구성단어 10개로 설정 

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [ ]:
RATING = 'after'
topics = model.print_topics(num_words = NUM_WORDS)
print_topic_prop(topics, RATING)

,topic_num,word_prop
0,1,"0.082*""푸루푸루"" + 0.073*""아기"" + 0.067*""않"" + 0.065*..."
1,2,"0.084*""말"" + 0.061*""푸루푸루"" + 0.037*""하마"" + 0.029*..."
2,3,"0.079*""푸루푸루"" + 0.053*""꾸루꾸루"" + 0.037*""않"" + 0.02..."
3,4,"0.115*""고마워"" + 0.059*""만들"" + 0.059*""올렸"" + 0.059*..."
4,5,"0.099*""아기"" + 0.057*""하마"" + 0.056*""푸루푸루"" + 0.045..."
5,6,"0.127*""알"" + 0.077*""푸루푸루"" + 0.059*""말"" + 0.045*""..."
6,7,"0.042*""내리쬐"" + 0.042*""비"" + 0.042*""조그만"" + 0.042*..."
7,8,"0.006*""떠나"" + 0.006*""보이"" + 0.006*""갑자기"" + 0.006*..."
8,9,"0.089*""하마"" + 0.087*""물"" + 0.078*""꾸루꾸루"" + 0.026*..."
9,10,"0.006*""떠나"" + 0.006*""보이"" + 0.006*""갑자기"" + 0.006*..."


In [ ]:
pyLDAvis.enable_notebook()
result_visualized = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
pyLDAvis.display(result_visualized)


In [ ]:
 # 시각화 결과 저장
RESULT_FILE = '/content/gdrive/MyDrive/result_nlp/' + RATING + '.html'
pyLDAvis.save_html(result_visualized, RESULT_FILE)

In [ ]:
corpus

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 2),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 3),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 2),
  (24, 1),
  (25, 2)],
 [(1, 1),
  (3, 1),
  (5, 1),
  (6, 1),
  (7, 3),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 4),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (21, 1),
  (22, 1),
  (23, 4),
  (25, 2),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1)],
 [(7, 4),
  (23, 2),
  (25, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1)],
 [(7, 1),
  (10, 2),
  (11, 1),
  (13, 1),
  (23, 1),
  (39, 1),
  (45, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1)],
 [(10, 2),
  (11, 4),
  (13, 2),
  (17, 1),
  (23, 3),
  (25, 2),
  (

### 4) 문서 별 토픽 분포 보기

위에서 토픽 별 단어 분포는 확인하였으나, 아직 문서 별 토픽 분포에 대해서는 확인하지 못 하였습니다. 우선 문서 별 토픽 분포를 확인하는 방법을 보겠습니다. 각 문서의 토픽 분포는 이미 훈련된 LDA 모델인 ldamodel[]에 전체 데이터가 정수 인코딩 된 결과를 넣은 후에 확인이 가능합니다. 여기서는 책의 지면의 한계로 상위 5개의 문서에 대해서만 토픽 분포를 확인해보겠습니다.

In [ ]:
for i, topic_list in enumerate(model[corpus]):
    if i==70:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(1, 0.20257184), (5, 0.772426)]
1 번째 문서의 topic 비율은 [(5, 0.9756743)]
2 번째 문서의 topic 비율은 [(1, 0.9608674)]
3 번째 문서의 topic 비율은 [(4, 0.70543057), (5, 0.24750814)]
4 번째 문서의 topic 비율은 [(4, 0.9756743)]
5 번째 문서의 topic 비율은 [(1, 0.28773013), (4, 0.69175434)]
6 번째 문서의 topic 비율은 [(1, 0.9006602), (5, 0.07433901)]
7 번째 문서의 topic 비율은 [(1, 0.97631484)]
8 번째 문서의 topic 비율은 [(0, 0.5832959), (1, 0.3859323)]
9 번째 문서의 topic 비율은 [(0, 0.96249825)]
10 번째 문서의 topic 비율은 [(0, 0.6039961), (6, 0.36400142)]
11 번째 문서의 topic 비율은 [(2, 0.97272444)]
12 번째 문서의 topic 비율은 [(2, 0.97631484)]
13 번째 문서의 topic 비율은 [(2, 0.4194457), (8, 0.5605532)]
14 번째 문서의 topic 비율은 [(8, 0.977499)]
15 번째 문서의 topic 비율은 [(3, 0.316237), (8, 0.65413123)]
16 번째 문서의 topic 비율은 [(3, 0.91817355)]
17 번째 문서의 topic 비율은 [(5, 0.97272587)]
18 번째 문서의 topic 비율은 [(1, 0.32370496), (5, 0.653437)]


In [ ]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [ ]:
topictable = make_topictable_per_doc(model, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호','가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:]



,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,5.0,0.7724,"[(1, 0.2025874), (5, 0.7724105)]"
1,1,5.0,0.9757,"[(5, 0.9756743)]"
2,2,1.0,0.9609,"[(1, 0.96086735)]"
3,3,4.0,0.7054,"[(4, 0.705436), (5, 0.24750268)]"
4,4,4.0,0.9757,"[(4, 0.9756743)]"
5,5,4.0,0.6918,"[(1, 0.28770936), (4, 0.691775)]"
6,6,1.0,0.9007,"[(1, 0.9006629), (5, 0.07433636)]"
7,7,1.0,0.9763,"[(1, 0.97631484)]"
8,8,0.0,0.5833,"[(0, 0.583331), (1, 0.385897)]"
9,9,0.0,0.9625,"[(0, 0.96249825)]"


In [ ]:
import os
path = '/content/gdrive/MyDrive/subjects2/'
file_list_hippo_py = []
for (root, dirs, files) in os.walk(path):
  if len(files) > 0:
    for file_name in files:
      if "hippo" in file_name:    
        file_list_hippo_py.append(root+'/'+file_name)
documents = []
for i in file_list_hippo_py:
  a=''
  for line in open((i),"r").readlines():
      if line and not line.replace(':','').replace(' ', '').strip().isdecimal():
        a=a+line.strip()
      else:
        if a!='':
          documents.append(a)
          a=''
  documents.append(a)
        
documents

['이야기 시작하겠습니다. 한 초원에서 있었던 일이야 그 초원에는 타조가 살았는데 그 타조는 알을 품고 있었어 그런데 그런데 그렇게 품에서 잠이 빠진 어느 날 타조 알 하나가 흘러들어가서는 하마이고 가버린 거야',
 '그걸 그 하마를 보고 사람들은 너도 새끼를 가졌구나 축하해 다른 동물들이 그렇게 축하를 해주고 있었지만 하마는 자고 있어서 깨어나고서야 자기가 알이 생겼다는 걸 알게 되었지 하마는 그래도 그게',
 '자기가 낳은 거라고 생각하게 되었고 자기 자식처럼 품고 지내고자 했어 그걸 보고 다른 펠리컨이 둥지를 만들어 주고 입안에 둥지를 만들어주면 안전할 거라고 말해줬어 그래서 하마는 자기 입안에다가 둥지를 만들고 그 둥지에 알을 품게 했지',
 '그렇게 며칠 고생 며칠 동안 아것도 못 먹고 아무 말도 할 수 없는 그런 고생을 했지 그러다가 며칠 뒤에 드디어 아래에서 타조가 깨어났어 그 사마 이름이 푸르푸르였는데 그 타조에게도 꾸루꾸루라는 이름을 붙여줬지 처음.태어났을 때 하마는 놀랐어 세상에 내가 낳은 자식이라는데 이렇게 닮았다고',
 '이런 충격이 있긴 했지만 그래도 이 자식이라 생각하고 열심히 키우기로 했지 오히려 다른 개구리들은 아마 얘는 커가면서 아마 너를 많이 닮아갈 거야 라고 말을 해줬어 그 말을 듣고 열심히 키워줬지 몇 년 동안 열심히 키웠지 물론 타조고 화마다 보니까',
 '그게 달라지진 않았어 그냥 그대로 익었고 딱히 닮은 건 없었지 그렇게 시간이 흘렀어 그렇게 타조가 큰 다음의 일이었어 갑자기 가뭄이 들어서 타조 하마들의 터전인 무릉덩이가 점점 줄어들고 있었어',
 '그걸 보다가 갑자기 타조인 꾸루꾸르가 그 타마를 곁을 떠나서 며칠 동안 열심히 달려 나가기 시작했어',
 '뿔을 부르는 그걸 보면서 드디어 얘도 다 성장해서 가출을 해버리는구나 집에 나가는구나라고 생각하고 안 돌아오겠지라고 생각을 했어 그러다 며칠이 지났어 놀랍게도 다시 타조가 돌아오기 시작한 거야 그러면서 제가 무릉덩이를 찾았어요.무릉덩이를 찾았다고 말을 하는 거 있지 그

In [ ]:
#데이터 전처리
hippo_df2 = pd.DataFrame({'document':documents})
hippo_df2

,document
0,이야기 시작하겠습니다. 한 초원에서 있었던 일이야 그 초원에는 타조가 살았는데 그 ...
1,그걸 그 하마를 보고 사람들은 너도 새끼를 가졌구나 축하해 다른 동물들이 그렇게 축...
2,자기가 낳은 거라고 생각하게 되었고 자기 자식처럼 품고 지내고자 했어 그걸 보고 다...
3,그렇게 며칠 고생 며칠 동안 아것도 못 먹고 아무 말도 할 수 없는 그런 고생을 했...
4,이런 충격이 있긴 했지만 그래도 이 자식이라 생각하고 열심히 키우기로 했지 오히려 ...
...,...
65,그래서 처음에는 푸르푸르가 이 아가가 태어난 아가가 이제 이름을 꾸루꾸르라고 지어주...
66,푸르푸루 너의 아이인 꾸루꾸루도 점점 크면서 너를 닮아갈 거야 이렇게 말씀을 해 주...
67,그러던 와중에 이제 하마들은 물이 있는 데서 항상 살아야 하는데 가뭄이 들어서 물이...
68,그래서 이제 대장 하마가 우리 여기 물이 없으니까 여기 있다가는 우리 모두 다 죽게...


In [ ]:
# 한글 외 텍스트 제거
#숫자, 특수문자, 영문 등  관련된 의미를 추출해 내기 어려운 모든 문자열을 제거합니다.
_removed = list(map(lambda text: re.sub('[^가-힣 ]', '', text), hippo_df2['document']))
hippo_df2['document'] = _removed
hippo_df2

,document
0,이야기 시작하겠습니다 한 초원에서 있었던 일이야 그 초원에는 타조가 살았는데 그 타...
1,그걸 그 하마를 보고 사람들은 너도 새끼를 가졌구나 축하해 다른 동물들이 그렇게 축...
2,자기가 낳은 거라고 생각하게 되었고 자기 자식처럼 품고 지내고자 했어 그걸 보고 다...
3,그렇게 며칠 고생 며칠 동안 아것도 못 먹고 아무 말도 할 수 없는 그런 고생을 했...
4,이런 충격이 있긴 했지만 그래도 이 자식이라 생각하고 열심히 키우기로 했지 오히려 ...
...,...
65,그래서 처음에는 푸르푸르가 이 아가가 태어난 아가가 이제 이름을 꾸루꾸르라고 지어주...
66,푸르푸루 너의 아이인 꾸루꾸루도 점점 크면서 너를 닮아갈 거야 이렇게 말씀을 해 주...
67,그러던 와중에 이제 하마들은 물이 있는 데서 항상 살아야 하는데 가뭄이 들어서 물이...
68,그래서 이제 대장 하마가 우리 여기 물이 없으니까 여기 있다가는 우리 모두 다 죽게...


In [ ]:
hippo_df2['document_after'] = ''
text_replaced_list = []
for text in tqdm(hippo_df2['document']):
    text_replaced = replace_word(str(text)) # 문자열 데이터 변환
    text_replaced_list.append(text_replaced)
hippo_df2['document_after'] = text_replaced_list
hippo_df2.head()

100%|██████████| 70/70 [00:00<00:00, 525.47it/s]


,document,document_after
0,이야기 시작하겠습니다 한 초원에서 있었던 일이야 그 초원에는 타조가 살았는데 그 타...,이야기 시작하겠습니다 한 초원에서 있었던 일이야 그 초원에는 타조가 살았는데 그 타...
1,그걸 그 하마를 보고 사람들은 너도 새끼를 가졌구나 축하해 다른 동물들이 그렇게 축...,그걸 그 하마를 보고 사람들은 너도 새끼를 가졌구나 축하해 다른 동물들이 그렇게 축...
2,자기가 낳은 거라고 생각하게 되었고 자기 자식처럼 품고 지내고자 했어 그걸 보고 다...,자기가 낳은 거라고 생각하게 되었고 자기 자식처럼 품고 지내고자 했어 그걸 보고 다...
3,그렇게 며칠 고생 며칠 동안 아것도 못 먹고 아무 말도 할 수 없는 그런 고생을 했...,그렇게 며칠 고생 며칠 동안 아것도 못 먹고 아무 말도 할 수 없는 그런 고생을 했...
4,이런 충격이 있긴 했지만 그래도 이 자식이라 생각하고 열심히 키우기로 했지 오히려 ...,이런 충격이 있긴 했지만 그래도 이 자식이라 생각하고 열심히 키우기로 했지 오히려 ...


In [ ]:
hippo_tokenized = list(map(lambda text: mecab.morphs(text), hippo_df2['document_after']))
hippo_tokenized

[['이야기',
  '시작',
  '하',
  '겠',
  '습니다',
  '한',
  '초원',
  '에서',
  '있',
  '었',
  '던',
  '일',
  '이',
  '야',
  '그',
  '초원',
  '에',
  '는',
  '타조',
  '가',
  '살',
  '았',
  '는데',
  '그',
  '타조',
  '는',
  '알',
  '을',
  '품',
  '고',
  '있',
  '었',
  '어',
  '그런데',
  '그런데',
  '그렇게',
  '품',
  '에서',
  '잠',
  '이',
  '빠진',
  '어느',
  '날',
  '타조',
  '알',
  '하나',
  '가',
  '흘러',
  '들어가',
  '서',
  '는',
  '하마',
  '이',
  '고',
  '가',
  '버린',
  '거',
  '야'],
 ['그걸',
  '그',
  '하마',
  '를',
  '보',
  '고',
  '사람',
  '들',
  '은',
  '너',
  '도',
  '새끼',
  '를',
  '가졌',
  '구나',
  '축하',
  '해',
  '다른',
  '동물',
  '들',
  '이',
  '그렇게',
  '축하',
  '를',
  '해',
  '주',
  '고',
  '있',
  '었',
  '지만',
  '하',
  '마는',
  '자',
  '고',
  '있',
  '어서',
  '깨어나',
  '고서',
  '야',
  '자기',
  '가',
  '알',
  '이',
  '생겼',
  '다는',
  '걸',
  '알',
  '게',
  '되',
  '었',
  '지',
  '하',
  '마는',
  '그래도',
  '그게'],
 ['자기',
  '가',
  '낳',
  '은',
  '거',
  '라고',
  '생각',
  '하',
  '게',
  '되',
  '었',
  '고',
  '자기',
  '자식',
  '처럼',
  '품',
  '고',
  '지내',
  '고자',
  '했',
  '어',


In [ ]:
text_removed_stopword = list(map(lambda tokens : remove_stopword(tokens),hippo_tokenized))
text_removed_stopword

[['이야기',
  '초원',
  '초원',
  '타조',
  '타조',
  '알',
  '품',
  '그런데',
  '그런데',
  '그렇게',
  '품',
  '빠진',
  '타조',
  '알',
  '흘러',
  '들어가',
  '하마',
  '버린'],
 ['하마',
  '새끼',
  '가졌',
  '축하',
  '동물',
  '그렇게',
  '축하',
  '마는',
  '깨어나',
  '알',
  '생겼',
  '다는',
  '알',
  '마는'],
 ['낳',
  '자식',
  '품',
  '지내',
  '고자',
  '펠리컨',
  '둥지',
  '만들',
  '입안',
  '둥지',
  '만들',
  '안',
  '전할',
  '말',
  '줬',
  '하마',
  '입안',
  '에다가',
  '둥지',
  '만들',
  '둥지',
  '알',
  '품'],
 ['그렇게',
  '먹',
  '말',
  '뒤',
  '드디어',
  '타조',
  '깨어났',
  '사마',
  '이름',
  '푸루푸루',
  '타조',
  '꾸루꾸루',
  '이름',
  '붙여',
  '줬',
  '태어났',
  '하마',
  '놀랐',
  '낳',
  '자식',
  '이렇게'],
 ['충격', '자식', '키우', '개구리', '아마', '아마', '말', '줬', '말', '키워', '줬', '키웠', '타조'],
 ['달라',
  '않',
  '그대로',
  '딱히',
  '그렇게',
  '흘렀',
  '그렇게',
  '타조',
  '갑자기',
  '가뭄',
  '타조',
  '하마',
  '터전',
  '물웅덩이',
  '점점',
  '줄어들'],
 ['갑자기', '타조', '하마', '떠나', '달려', '나가'],
 ['부르',
  '드디어',
  '성장',
  '가출',
  '버리',
  '는구나',
  '나가',
  '는구나',
  '안',
  '돌아오',
  '지났',
  '놀랍',
  '다시',
  '타조',
  '돌아오',
  '물웅덩이',
 

In [ ]:
corpus = [dictionary.doc2bow(text) for text in text_removed_stopword]
corpus

[[(13, 2), (20, 3), (24, 2), (25, 1), (56, 2)],
 [(5, 1), (13, 2), (19, 2), (25, 1), (73, 1)],
 [(7, 1),
  (13, 1),
  (24, 2),
  (25, 1),
  (26, 1),
  (69, 1),
  (74, 4),
  (76, 3),
  (90, 1)],
 [(7, 1),
  (20, 2),
  (23, 1),
  (25, 1),
  (26, 1),
  (39, 1),
  (41, 1),
  (91, 1),
  (94, 2),
  (136, 1)],
 [(7, 2), (20, 1), (66, 1)],
 [(20, 2), (25, 1), (43, 1), (103, 1), (111, 1)],
 [(20, 1), (25, 1), (35, 1), (111, 1), (118, 1)],
 [(7, 1), (20, 1), (35, 1), (38, 2), (69, 1), (99, 1), (103, 2), (115, 1)],
 [(7, 1), (20, 1), (25, 3), (91, 1), (103, 1), (118, 1), (161, 1)],
 [(25, 1), (115, 1)],
 [(5, 1),
  (13, 3),
  (19, 1),
  (20, 1),
  (23, 2),
  (24, 1),
  (25, 2),
  (26, 1),
  (145, 1)],
 [],
 [(8, 2),
  (13, 3),
  (23, 2),
  (24, 1),
  (49, 1),
  (60, 1),
  (74, 1),
  (76, 1),
  (87, 2)],
 [(7, 1), (13, 2), (23, 4), (25, 3), (37, 1), (39, 1), (56, 2), (87, 1)],
 [(23, 1), (25, 1), (56, 1), (60, 1), (66, 1), (92, 1)],
 [(23, 1),
  (25, 3),
  (43, 1),
  (60, 1),
  (91, 1),
  (105, 1)

In [ ]:
for i, topic_list in enumerate(model[corpus]):
    if i==70:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(1, 0.50333697), (4, 0.4239293)]
1 번째 문서의 topic 비율은 [(0, 0.012500849), (1, 0.14978515), (2, 0.01250011), (3, 0.012500444), (4, 0.012501266), (5, 0.75021094), (6, 0.012500001), (7, 0.012500001), (8, 0.012501231), (9, 0.012500001)]
2 번째 문서의 topic 비율은 [(1, 0.8028307), (5, 0.14716603)]
3 번째 문서의 topic 비율은 [(0, 0.32348678), (1, 0.3679962), (5, 0.13534309), (8, 0.12701848)]
4 번째 문서의 topic 비율은 [(0, 0.02), (1, 0.819997), (2, 0.02), (3, 0.02), (4, 0.02), (5, 0.02000229), (6, 0.02), (7, 0.020000001), (8, 0.020000698), (9, 0.020000001)]
5 번째 문서의 topic 비율은 [(0, 0.014289668), (1, 0.489513), (2, 0.39618072), (3, 0.014286833), (4, 0.014288597), (5, 0.014287474), (6, 0.01429314), (7, 0.014285716), (8, 0.014289096), (9, 0.014285716)]
6 번째 문서의 topic 비율은 [(0, 0.016669668), (1, 0.49364793), (2, 0.37300494), (3, 0.016668236), (4, 0.016669359), (5, 0.01666882), (6, 0.016666668), (7, 0.016666668), (8, 0.016671011), (9, 0.016666668)]
7 번째 문서의 topic 비율은 [(0, 0.09998799), (1, 0.53111744), (2,

In [ ]:
topictable = make_topictable_per_doc(model, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,1.0,0.5033,"[(1, 0.50332), (4, 0.42394638)]"
1,1,5.0,0.7502,"[(0, 0.0125008505), (1, 0.14978267), (2, 0.012..."
2,2,1.0,0.8028,"[(1, 0.8028298), (5, 0.147167)]"
3,3,1.0,0.3679,"[(0, 0.32357723), (1, 0.36794013), (5, 0.13536..."
4,4,1.0,0.8200,"[(0, 0.02), (1, 0.819997), (2, 0.02), (3, 0.02..."
...,...,...,...,...
65,65,0.0,0.2568,"[(0, 0.25681025), (1, 0.20300479), (2, 0.01250..."
66,66,0.0,0.6733,"[(0, 0.67332566), (1, 0.124771684), (5, 0.1382..."
67,67,2.0,0.4352,"[(1, 0.1003857), (2, 0.43521553), (8, 0.410547..."
68,68,8.0,0.6895,"[(2, 0.25331467), (8, 0.68953264)]"


In [ ]:
hippo_df2['document_after'][0]

'이야기 시작하겠습니다 한 초원에서 있었던 일이야 그 초원에는 타조가 살았는데 그 타조는 알을 품고 있었어 그런데 그런데 그렇게 품에서 잠이 빠진 어느 날 타조 알 하나가 흘러들어가서는 하마이고 가버린 거야'

In [ ]:
def make_topictable_per_doc2(ldamodel, corpus,document):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([document['document_after'][i],int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [ ]:
topictable = make_topictable_per_doc2(model, corpus,hippo_df2)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호','document', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:]

,문서 번호,document,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,이야기 시작하겠습니다 한 초원에서 있었던 일이야 그 초원에는 타조가 살았는데 그 타...,1.0,0.5034,"[(1, 0.5033559), (4, 0.42390984)]"
1,1,그걸 그 하마를 보고 사람들은 너도 새끼를 가졌구나 축하해 다른 동물들이 그렇게 축...,5.0,0.7503,"[(0, 0.012500848), (1, 0.14974011), (2, 0.0125..."
2,2,자기가 낳은 거라고 생각하게 되었고 자기 자식처럼 품고 지내고자 했어 그걸 보고 다...,1.0,0.8028,"[(1, 0.8028304), (5, 0.14716634)]"
3,3,그렇게 며칠 고생 며칠 동안 아것도 못 먹고 아무 말도 할 수 없는 그런 고생을 했...,1.0,0.3679,"[(0, 0.32346815), (1, 0.36793107), (5, 0.13540..."
4,4,이런 충격이 있긴 했지만 그래도 이 자식이라 생각하고 열심히 키우기로 했지 오히려 ...,1.0,0.8200,"[(0, 0.020000001), (1, 0.819997), (2, 0.020000..."
...,...,...,...,...,...
65,65,그래서 처음에는 푸루푸루가 이 아가가 태어난 아가가 이제 이름을 꾸루꾸르라고 지어주...,0.0,0.2568,"[(0, 0.2567567), (1, 0.20322168), (2, 0.012502..."
66,66,푸루푸루 너의 아이인 꾸루꾸루도 점점 크면서 너를 닮아갈 거야 이렇게 말씀을 해 주...,0.0,0.6733,"[(0, 0.6733217), (1, 0.124772355), (5, 0.13826..."
67,67,그러던 와중에 이제 하마들은 물이 있는 데서 항상 살아야 하는데 가뭄이 들어서 물이...,2.0,0.4352,"[(1, 0.10040964), (2, 0.43515304), (8, 0.41058..."
68,68,그래서 이제 대장 하마가 우리 여기 물이 없으니까 여기 있다가는 우리 모두 다 죽게...,8.0,0.6895,"[(2, 0.2533152), (8, 0.68953216)]"
